In [4]:
import numpy as np
import pandas as pd
import pyarrow.feather as feather
import plotly.express as px

In [118]:
df = feather.read_table(r"C:\Users\mexic\Documents\Python\data\cumulative_2006-2022.feather").to_pandas()

#df.head()

After importing data, we will focus on tracking changes on a state to state level across all years in the sample.

In [148]:
#add state abbbv column, ie FL CA etc
####how to convert fips codes to state abbv ie FL, GA
fips_to_state = {
    1: 'AL', 2: 'AK', 4: 'AZ', 5: 'AR', 6: 'CA', 8: 'CO', 9: 'CT', 10: 'DE',
    11: 'DC', 12: 'FL', 13: 'GA', 15: 'HI', 16: 'ID', 17: 'IL', 18: 'IN',
    19: 'IA', 20: 'KS', 21: 'KY', 22: 'LA', 23: 'ME', 24: 'MD', 25: 'MA',
    26: 'MI', 27: 'MN', 28: 'MS', 29: 'MO', 30: 'MT', 31: 'NE', 32: 'NV',
    33: 'NH', 34: 'NJ', 35: 'NM', 36: 'NY', 37: 'NC', 38: 'ND', 39: 'OH',
    40: 'OK', 41: 'OR', 42: 'PA', 44: 'RI', 45: 'SC', 46: 'SD', 47: 'TN',
    48: 'TX', 49: 'UT', 50: 'VT', 51: 'VA', 53: 'WA', 54: 'WV', 55: 'WI',
    56: 'WY'
}

df['state_abbv'] = df['state'].map(fips_to_state)

# remove NAs
df_sex = df[df['sex'].notna()].copy()

#convert to categorical Male/Female
df_sex['sex2'] = df_sex['sex'].replace({1.0: "Male",
                                        2.0: "Female"})


# Group by 'Year', 'State_abbv', and 'Sex' and count the number of each sex in each group
count_table = df_sex.groupby(['year', 'state_abbv', 'sex2']).size().reset_index(name='Count')

# Pivot the table to have 'Year' and 'State_abbv' as the index, 'Sex' as columns, and 'Count' as values
pivot_table = count_table.pivot_table(index=['year', 'state_abbv'], columns='sex2', values='Count', fill_value=0)

# Calculate the proportion of Male and Female
pivot_table['Male Proportion'] = pivot_table['Male'] / (pivot_table['Male'] + pivot_table['Female']) * 100
pivot_table['Female Proportion'] = pivot_table['Female'] / (pivot_table['Male'] + pivot_table['Female']) * 100

# Reset the index to have 'Year' and 'State_abbv' as columns
pivot_table.reset_index(inplace=True)

# Display the resulting table
#print(pivot_table)


sex2             Female    Male
year state_abbv                
2006 AK            75.0    56.0
     AL           311.0   184.0
     AR           199.0   157.0
     AZ           471.0   492.0
     CA          1801.0  1836.0
...                 ...     ...
2020 VT            85.0    65.0
     WA           728.0   646.0
     WI           716.0   518.0
     WV           261.0   183.0
     WY            55.0    40.0

[765 rows x 2 columns]


KeyError: 'Very Liberal'

In [159]:
## Create poportion table for ideology

#filter out NAs from ideology respondants
df_ideo = df[df['ideo5'].notna()].copy()
#print(df_ideo['ideo5'].unique)

# Group by 'Year', 'State_abbv', and 'Sex' and count the number of each sex in each group
count_table = df_ideo.groupby(['year', 'state_abbv', 'ideo5']).size().reset_index(name='Count')

# Pivot the table to have 'Year' and 'State_abbv' as the index, 'Sex' as columns, and 'Count' as values
pivot_table = count_table.pivot_table(index=['year', 'state_abbv'], columns='ideo5', values='Count', fill_value=0)
#print(pivot_table)
# Calculate the proportion of ideologies
categories = ['Very Liberal', 'Liberal', 'Moderate', 'Conservative', 'Very Conservative', 'Not Sure']  # List of all possible categories
for category in categories:
    pivot_table[f'{category} Proportion'] = pivot_table[category] / pivot_table[categories].sum(axis=1) * 100


# Reset the index to have 'Year' and 'State_abbv' as columns
pivot_table.reset_index(inplace=True)

#test if result worked by adding proportion from a given year and state
print(pivot_table[(pivot_table['year'] == 2012) & (pivot_table['state_abbv'] == 'FL')])

ideo5  year state_abbv  Very Liberal  Liberal  Moderate  Conservative  \
315    2012         FL         245.0    646.0    1222.0         997.0   

ideo5  Very Conservative  Not Sure  Very Liberal Proportion  \
315                460.0     273.0                 6.375228   

ideo5  Liberal Proportion  Moderate Proportion  Conservative Proportion  \
315             16.809784            31.798074                25.943273   

ideo5  Very Conservative Proportion  Not Sure Proportion  
315                       11.969815             7.103825  


C:\Users\mexic\AppData\Local\Temp\ipykernel_9876\4190988424.py:6: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [163]:
# Create an animated choropleth map using Plotly
fig = px.choropleth(
    pivot_table,
    locations="state_abbv",
    locationmode="USA-states",
    color="Very Conservative Proportion",
    animation_frame="year",  # Specify the column for animation
    scope="usa",
    color_continuous_scale="Reds",
    title="Choropleth Map Over Years",
    range_color=(pivot_table['Very Conservative Proportion'].min(), pivot_table['Very Conservative Proportion'].max())  # Set the color scale range
)

fig.update_geos(
    resolution=50,
    showcoastlines=True,
    coastlinecolor="Black",
    showland=True,
    landcolor="white"
)

fig.show()